In [1]:
import numpy as np

import sys
sys.path.append("../")
from koopmanlib.dictionary import PsiNN

In [2]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=100)

In [3]:
from duffing_oscillator import duffing_oscillator
from scipy.integrate import solve_ivp

# Parameters
delta = 0.5
alpha = -1
beta = 1

# Sample N=1000 initial points, with uniform distribution [-2, 2].
N = 1000
np.random.seed(39)
sample_data_x0 = (np.random.rand(N, 2) * 4) - 2

tspan = [0, 2.5]
nstep = 11
data = np.zeros((N, 2, nstep))

t_eval = np.linspace(tspan[0], tspan[1], nstep)

for idx, x0 in enumerate(sample_data_x0):
    sol = solve_ivp(duffing_oscillator, tspan, x0, t_eval=t_eval, args=(delta, alpha, beta))
    data[idx, 0, :] = sol.y[0]
    data[idx, 1, :] = sol.y[1]

# Pairwise shifted data X and Y
X = data[:, :, :-1] # shape: (1000, 2, 10)
Y = data[:, :, 1:]
print(X.shape, type(X))
print(Y.shape, type(Y))

# Transpose and reshape the data.
X = np.transpose(X, (0, 2, 1)).reshape(N * (nstep-1), 2) # shape: (10000, 2)
Y = np.transpose(Y, (0, 2, 1)).reshape(N * (nstep-1), 2)
print(X.shape, type(X))
print(Y.shape, type(Y))

(1000, 2, 10) <class 'numpy.ndarray'>
(1000, 2, 10) <class 'numpy.ndarray'>
(10000, 2) <class 'numpy.ndarray'>
(10000, 2) <class 'numpy.ndarray'>


In [4]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [5]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [6]:
from koopmanlib.solver_4 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node 'model/concatenate_2/concat' defined at (most recent call last):
    File "c:\Users\Jason\miniconda3\envs\koopman\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Jason\miniconda3\envs\koopman\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Jason\miniconda3\envs\koopman\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Jason\miniconda3\envs\koopman\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\Jason\miniconda3\envs\koopman\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Jason\miniconda3\envs\koopman\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Jason\m

In [ ]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs4 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs4['resDMD_DL_outputs']['efuns'].shape)

In [ ]:
from koopmanlib.solver_0 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs0 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs0['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs0['resDMD_DL_outputs']['efuns'].shape)

In [ ]:
from koopmanlib.solver_1 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs1 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs1['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs1['resDMD_DL_outputs']['efuns'].shape)

In [ ]:
from koopmanlib.solver_2 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs2 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs2['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs2['resDMD_DL_outputs']['efuns'].shape)

In [ ]:
from koopmanlib.solver_3 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)